# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [43]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.pipeline.steps import AutoMLStep

from azureml.widgets import RunDetails

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-udacity'

experiment=Experiment(ws, experiment_name)


# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "comp-udacity"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=16)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.


In [38]:

from azureml.core import Workspace, Dataset, Datastore

subscription_id = 'bfdbbde2-f093-45f7-9b9d-c1721eed4782'
resource_group = 't.tschuemperlin-rg'
workspace_name = 'final-project-udacity'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "workspaceblobstore")
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'UI/2023-11-13_084243_UTC/heart.csv'))
df = dataset.to_pandas_dataframe()
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [39]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task="classification",
    training_data=dataset,
    label_column_name="output",
    enable_early_stopping=True,
    iterations=200,
    featurization='auto',
    **automl_settings
)

In [40]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on comp-udacity with default configuration
Running on remote compute: comp-udacity


Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity,AutoML_6622dc6d-999d-430a-a6fe-c73f87fc7ec2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [44]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

Bad pipe message: %s [b'\xf9\xd8\xf0[oB\xab\x0ca\xa7a\x99Q\xa4"NQ\x9d U\x81$\x17p:\xaf\xaf\xe1\xc8\x1f\xa3\xb5\xcc\xdb\xec\xab\x0e\x08\xbf\x8c=_&\x91\xbd\'\xf7\xd8\x91\x1dV\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 d<?tuT>\xe5\xd7\xe0<\xd6!V', b'']
Bad pipe message: %s [b'\xb3\xdf\xa3W\x12\x04\xa3\xbe\x0cai\xf5\xd7Fby:\xb6 \x93 \xc4\xf2\x01nT\xc9\x0c\xf6\xb7\xdaR:\x1f:\xc0\xec\x1b\xfa\xf5!g\xc4;\xf9\xac\x13\x04TeS\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t12']
Bad pipe message: %s [b'0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity,AutoML_6622dc6d-999d-430a-a6fe-c73f87fc7ec2,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|10                            |
+------

{'runId': 'AutoML_6622dc6d-999d-430a-a6fe-c73f87fc7ec2',
 'target': 'comp-udacity',
 'status': 'Completed',
 'startTimeUtc': '2023-11-13T10:14:23.664071Z',
 'endTimeUtc': '2023-11-13T10:36:41.252639Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '200',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'comp-udacity',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-udacity","subscription_id":"bfdbbde2-f093-45f7-9b9d-c1721eed4782","resource_group":"t.tschuemperlin-rg","workspace_name":"final-project-udacity","region":"eastus","compute_target":"comp-udacity","spark_servi

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [48]:
# Get best run and model
best_run, model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
display(best_run)
print("Accuracy: ", best_run_metrics['accuracy'])
print(model)
print(best_run_metrics)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity,AutoML_6622dc6d-999d-430a-a6fe-c73f87fc7ec2_48,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


Accuracy:  0.857741935483871
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('29', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features=0.1, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.10368421052631578, min_weight_fraction_leaf=0.0, n_estimators=200, n_job

In [58]:
best_run.get_metrics()

{'f1_score_macro': 0.8516445266952216,
 'balanced_accuracy': 0.8522108041167638,
 'AUC_macro': 0.9161249861106672,
 'weighted_accuracy': 0.8627354638958964,
 'average_precision_score_weighted': 0.9221583024926616,
 'average_precision_score_micro': 0.9148667857978486,
 'average_precision_score_macro': 0.9181821120930275,
 'precision_score_micro': 0.857741935483871,
 'precision_score_weighted': 0.8621269409397966,
 'precision_score_macro': 0.8586273816053227,
 'recall_score_macro': 0.8522108041167638,
 'log_loss': 0.37686764347601365,
 'recall_score_weighted': 0.857741935483871,
 'norm_macro_recall': 0.7044216082335277,
 'AUC_weighted': 0.9161249861106672,
 'accuracy': 0.857741935483871,
 'recall_score_micro': 0.857741935483871,
 'AUC_micro': 0.9133665163602729,
 'f1_score_weighted': 0.856403591630774,
 'matthews_correlation': 0.7105484510772695,
 'f1_score_micro': 0.857741935483871,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_6622dc6d-999d-430a-a6fe-c73f87fc7ec2_48/

In [59]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [47]:
#TODO: Save the best model
best_model = best_run.register_model(
    model_path='./outputs/model.pkl',
    model_name='best_automl_model', properties=dict(best_run_metrics)
)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [49]:
from azureml.core.model import InferenceConfig
from azureml.train.automl import constants
from azureml.automl.core.shared import constants
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model



#deploying the already registed model (best), creating an inference and --> model exposed as a WS-*.
#getting environment from best run
env = best_run.get_environment()

#register the model
model = remote_run.register_model(model_name=best_run.properties['model_name'])


best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(
                                                       cpu_cores=1,
                                                       memory_gb=1,
                                                       description='heart dataset',
                                                       auth_enabled=True,
                                                       enable_app_insights= True,
                                                       collect_model_data = True)

service = Model.deploy(ws, 'heart-failure-service', [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-13 11:02:20+00:00 Creating Container Registry if not exists..
2023-11-13 11:12:21+00:00 Registering the environment..
2023-11-13 11:12:22+00:00 Use the existing image.
2023-11-13 11:12:23+00:00 Submitting deployment to compute..
2023-11-13 11:12:28+00:00 Checking the status of deployment heart-failure-service..
2023-11-13 11:14:54+00:00 Checking the status of inference endpoint heart-failure-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [51]:
print('Score URI:',service.scoring_uri)
print('Swagger URL:',service.swagger_uri)
service.state

Score URI: http://4f4f169c-0012-4676-81c3-d62f2f8b07b4.eastus.azurecontainer.io/score
Swagger URL: http://4f4f169c-0012-4676-81c3-d62f2f8b07b4.eastus.azurecontainer.io/swagger.json


'Healthy'

TODO: In the cell below, print the logs of the web service and delete the service

In [52]:
print(service.get_logs())

2023-11-13T11:14:22,294665800+00:00 - rsyslog/run 
2023-11-13T11:14:22,303122300+00:00 - gunicorn/run 
2023-11-13T11:14:22,305650600+00:00 | gunicorn/run | 
2023-11-13T11:14:22,311294600+00:00 | gunicorn/run | ###############################################
2023-11-13T11:14:22,314068600+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-11-13T11:14:22,319335800+00:00 | gunicorn/run | ###############################################
2023-11-13T11:14:22,321204000+00:00 | gunicorn/run | 
2023-11-13T11:14:22,324369000+00:00 - nginx/run 
2023-11-13T11:14:22,329802500+00:00 | gunicorn/run | 
2023-11-13T11:14:22,343276800+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2023-11-13T11:14:22,346210900+00:00 | gunicorn/run | 
2023-11-13T11:14:22,353256100+00:00 | gunicorn/run | 
2023-11-13T11:14:22,367250900+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [54]:
service.delete()

No service with name heart-failure-service found to delete.


In [56]:
service.state

'Deleting'

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
